In [1]:
import sys
sys.path.append("..")
from utils.dataset import FerDataset

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
%matplotlib inline


In [9]:
class VGG13(nn.Module):
    
    def __init__(self):
        super(VGG13, self).__init__()
        
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 48, kernel_size=(3, 3), stride = 1, padding = 0),
            # kernel size F=3, stride S = 1, to retain input size padding must be P = (F - 1)/2
            nn.ReLU(),
# sollte man ReLU immer nach jeder Conv layer einfuegen?
            nn.Conv2d(48, 48, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            # max pool with F=2 and S=2 chooses the max out of a 2x2 square and only keeps that max value.
            # Therefore 75% of the information are left out
            # the max pool layer works on every depth dimension independently, therefore the input depth remains
            # unchanged
            nn.Dropout2d(p=0.25),
# im Paper steht "a drop out rate of 25%". Heisst, dass p=0.25 oder p=0.75
# und warum genau gibt es in dieser Architektur eine extra dropout layer? Normalerweise dachte ich, sollte jede conv
# layer eine dropout layer sein um overfitten zu reduzieren oder?
            
            
            nn.Conv2d(48, 96, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout2d(p=0.25),
            
            
            nn.Conv2d(96, 192, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.Conv2d(192, 192, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.Conv2d(192, 192, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout2d(p=0.25),
            
            
            nn.Conv2d(192, 192, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.Conv2d(192, 192, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.Conv2d(192, 192, kernel_size=(3, 3), stride = 1, padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout2d(p=0.25)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(576, 576),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(576, 576),
# ist die Linear layer eine fully connected layer? Und was bringen zwei davon hinterinenander?
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(576,10),
            nn.Softmax(10)
# ich verwenden unten die Funktionen von dir, also als loss funktion die softmax funktion. Muss ich dann trotzdem
# hier noch eine Softmax layer einfuegen? Weil du hast das nicht gemacht beim LeNet
        )
        
        
    def forward(self, x):
        x = self.convnet(x)
        x = x.view(x.size(0), -1)
# Was macht dieses funktion view()?
        x = self.fc(x)
        return x
            
            
        
        
        
            
            
            
            

In [10]:
dataset = FerDataset(base_path='../../data',
                     data='ferplus',
                     mode='train',
                     label='ferplus_votes')
dataloader = DataLoader(dataset, batch_size=24, shuffle=True, num_workers=0)
net = VGG13()
log_softmax = nn.LogSoftmax(dim=-1)
criterion = nn.KLDivLoss(size_average=False)
optimizer = torch.optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [11]:
x_batch, y_batch = next(iter(dataloader))
losses = []
for i in range(1000):
    optimizer.zero_grad()
    logits = net(x_batch)
    log_probs = log_softmax(logits)
    loss = criterion(log_probs, y_batch)
    losses.append(float(loss))
    loss.backward()
    optimizer.step()
    print(i, end='\r')

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't greater than actual input size at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/THNN/generic/SpatialConvolutionMM.c:48